In [1]:

%pip install  pandas numpy sentence-transformers scikit-learn requests

  Using cached pandas-2.3.1-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached scikit_learn-1.7.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.8.0-cp311-cp311-win_amd64.whl.metadata (30 kB)
  Using cached scipy-1.16.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached pillow-11.3.0-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached reg

# Import library



In [1]:
import requests
import pandas as pd
import numpy as np
import re
from requests.auth import HTTPBasicAuth
# from sentence_transformers import SentenceTransformer, util, CrossEncoder

Connection To local server

In [3]:
import requests
from requests.auth import HTTPBasicAuth

endpoint = 'Person'
api_url = f"http://localhost:1251/api/{endpoint}"  # f-string so endpoint is inserted
username = "support@uniphi.com.au"
password = "Luna@9860"

try:
    response = requests.get(api_url, auth=HTTPBasicAuth(username, password), timeout=10)
    print("Status Code:", response.status_code)
    print("Raw JSON Preview:")
    print(response.json())
except requests.exceptions.RequestException as e:
    print("Error connecting to API:", e)


Status Code: 200
Raw JSON Preview:
[{'ID': 59, 'Name': 'UniPhi User', 'ContactID': '', 'Email': 'support@uniphi.com.au', 'Phone': '1300 79 48 68', 'Address1': 'Suite 3.02 St Martins Tower', 'Address2': 'Level 3 31 Market St', 'Suburb': 'Sydney', 'State': 'NSW', 'Country': 'Australia', 'Postcode': '2000', 'Position': '', 'Login': 'support@uniphi.com.au', 'CurrencyID': 1, 'AccessID': 3, 'AccessName': 'Administrator', 'StandardHours': 8.0, 'Organisations': [{'ID': 1, 'Name': 'UniPhi Pty Ltd'}], 'Sites': [{'ID': 1, 'Name': 'UniPhi (Sydney)', 'OrganisationID': 1, 'Phone': '1300 79 48 68', 'Fax': '', 'CompanyNumber': '', 'TaxNumber': '', 'Address1': 'Suite 3.02 St Martins Tower', 'Address2': 'Level 3 31 Market St', 'Suburb': 'Sydney', 'State': 'NSW', 'Postcode': '2000', 'Country': 'Australia', 'IsDisabled': False, 'Email': '', 'BankName': '', 'BankCode': '', 'BankAccountName': '', 'BankAccountNumber': '', 'CustomFields': []}], 'IsDisabled': False, 'IsInternal': False, 'CustomFields': []}, {'

In [4]:
data=response.json()
df = pd.DataFrame(data)
df.head()

,ID,Name,ContactID,Email,Phone,Address1,Address2,Suburb,State,Country,...,ClassificationID,ClassificationCode,Classification,Multiplier,EmploymentType,Salary,ChargeRate,CostRate,StartDate,EndDate
0,59,UniPhi User,,support@uniphi.com.au,1300 79 48 68,Suite 3.02 St Martins Tower,Level 3 31 Market St,Sydney,NSW,Australia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2119,Rhianna Whitcombe,10725,rhianna.whitcombe@arcadis.com,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,ARC_S,Acradis PAA Salary,Arcadis Salary,Base Calc,212.85,68.11,212.85,2023-07-01,NaN
2,2120,Jon Yearsley,11999,jonathan.yearsley@arcadis.com,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,ARC_S,Acradis PAA Salary,Arcadis Salary,Base Calc,336.69,107.74,336.69,2024-08-12,NaN
3,2121,Sergio Fuentes,10605,Sergio.fuentes@arcadis.com,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,ARC_S,Acradis PAA Salary,Arcadis Salary,Base Calc,269.70,86.30,269.70,2024-08-12,NaN
4,2122,Richard Russell,11062,richard.russell@arcadis.com,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,ARC_S,Acradis PAA Salary,Arcadis Salary,Base Calc,308.68,98.78,308.68,2024-09-09,NaN


In [5]:
df_uniphiData=df[["ID","ClassificationID","ClassificationCode","Classification","Position"]].drop_duplicates()
df_uniphiData.head()


,ID,ClassificationID,ClassificationCode,Classification,Position
0,59,NaN,NaN,NaN,
1,2119,35.0,ARC_S,Acradis PAA Salary,Commercial Manager
2,2120,35.0,ARC_S,Acradis PAA Salary,Design Manager - North
3,2121,35.0,ARC_S,Acradis PAA Salary,Digital Engineering Discipline Lead
4,2122,35.0,ARC_S,Acradis PAA Salary,Innovation & Value Engineering Lead


Extract Customefield


In [6]:
def extract_discipline(custom_fields):
    if isinstance(custom_fields, list):
        for field in custom_fields:
            if field.get("Name") == "Discipline":
                return field.get("Value")
    return None
# Apply custom function to extract Discipline
df["Discipline"] = df["CustomFields"].apply(extract_discipline)

Extract data with classfication

In [7]:
df["Discipline"] = df["CustomFields"].apply(extract_discipline)
df_uniphiData=df[["ID","ClassificationID","ClassificationCode","Classification","Discipline","Position"]].drop_duplicates()
print(df_uniphiData.head())
df_uniphiData.to_csv("uniphi_person_classification.csv", index=False)


     ID  ClassificationID ClassificationCode      Classification  \
0    59               NaN                NaN                 NaN   
1  2119              35.0              ARC_S  Acradis PAA Salary   
2  2120              35.0              ARC_S  Acradis PAA Salary   
3  2121              35.0              ARC_S  Acradis PAA Salary   
4  2122              35.0              ARC_S  Acradis PAA Salary   

           Discipline                             Position  
0                None                                       
1  Project Management                   Commercial Manager  
2   Design Management               Design Manager - North  
3  Project Management  Digital Engineering Discipline Lead  
4  Project Management  Innovation & Value Engineering Lead  


Import Classficiation File 

In [8]:

df_main = pd.read_csv("C:/Users/SuratAdhikari/Desktop/classfication/Uniclass2015_Ro_v1_11.csv", header=2)
df_main.head()

,Code,Group,Sub group,Section,Object,Sub object,Title,NBS Code,COBie,NRM1,IFC 2x3 TC1,IFC 4Add2 TC1,IFC 4x3Add2 TC1,CESMM
0,Ro_10,10,NaN,NaN,NaN,NaN,Management roles,NaN,Ro_10 : Management roles,NaN,NaN,NaN,NaN,NaN
1,Ro_10_10,10,10.0,NaN,NaN,NaN,Organization roles,NaN,Ro_10_10 : Organization roles,NaN,NaN,NaN,NaN,NaN
2,Ro_10_10_02,10,10.0,2.0,NaN,NaN,Accountable person,NaN,Ro_10_10_02 : Accountable person,NaN,NaN,NaN,NaN,NaN
3,Ro_10_10_05,10,10.0,5.0,NaN,NaN,Associate,NaN,Ro_10_10_05 : Associate,NaN,NaN,NaN,NaN,NaN
4,Ro_10_10_12,10,10.0,12.0,NaN,NaN,Chairman,NaN,Ro_10_10_12 : Chairman,NaN,NaN,NaN,NaN,NaN


Extract only uniclassCode and uniClassTitile

In [9]:
df_uniclass=df_main[["Code","Title"]]
df_uniphiData.head(100)

,ID,ClassificationID,ClassificationCode,Classification,Discipline,Position
0,59,NaN,NaN,NaN,None,
1,2119,35.0,ARC_S,Acradis PAA Salary,Project Management,Commercial Manager
2,2120,35.0,ARC_S,Acradis PAA Salary,Design Management,Design Manager - North
3,2121,35.0,ARC_S,Acradis PAA Salary,Project Management,Digital Engineering Discipline Lead
4,2122,35.0,ARC_S,Acradis PAA Salary,Project Management,Innovation & Value Engineering Lead
...,...,...,...,...,...,...
95,2213,35.0,ARC_S,Acradis PAA Salary,Lighting,Lighting Engineer/Designer
96,2214,38.0,ARC_G,Arcadis GEC,Drafting,Principal CAD Technician
97,2215,35.0,ARC_S,Acradis PAA Salary,Structures,Cut and Cover Engineer - NTSP
98,2216,18.0,P24,Qualified drafter/modeller,Digital Engineering,Structures BIM Coordinator


#Rename

In [10]:
df_uniclass=df_uniclass.rename(columns={'Title':'UniclassTitle','Code':'UniclassCode'})
print(df_uniclass.count())

#Total 287 Unilcass role.


df_uniphiData.head()
df_uniphiData=df_uniphiData[2:]
df_uniphiData.head()

UniclassCode     287
UniclassTitle    287
dtype: int64


,ID,ClassificationID,ClassificationCode,Classification,Discipline,Position
2,2120,35.0,ARC_S,Acradis PAA Salary,Design Management,Design Manager - North
3,2121,35.0,ARC_S,Acradis PAA Salary,Project Management,Digital Engineering Discipline Lead
4,2122,35.0,ARC_S,Acradis PAA Salary,Project Management,Innovation & Value Engineering Lead
5,2123,35.0,ARC_S,Acradis PAA Salary,Design Management,AMT Design Lead
6,2124,35.0,ARC_S,Acradis PAA Salary,Building Structures,Buildings Structures Lead


In [11]:
df_uniclass.head()

,UniclassCode,UniclassTitle
0,Ro_10,Management roles
1,Ro_10_10,Organization roles
2,Ro_10_10_02,Accountable person
3,Ro_10_10_05,Associate
4,Ro_10_10_12,Chairman


Minimal prep on your two DataFrames

In [12]:
# Make safe copies
df_u = df_uniphiData.copy()
df_c = df_uniclass.copy()

# Fill blanks and strip whitespace
for col in ["Classification","Discipline","Position","ClassificationCode"]:
    if col in df_u.columns:
        df_u[col] = df_u[col].fillna("").astype(str).str.strip()
# .fillna("") --replaces any missing values (NaN) in the column with an empty string "
#.astype(str)--converts everything to a string type.
#.str.strip()--removes any extra spaces at the start or end of the text.
df_c["UniclassTitle"] = df_c["UniclassTitle"].fillna("").astype(str).str.strip()
df_c["UniclassCode"]  = df_c["UniclassCode"].fillna("").astype(str).str.strip()
df_c.head(250)

,UniclassCode,UniclassTitle
0,Ro_10,Management roles
1,Ro_10_10,Organization roles
2,Ro_10_10_02,Accountable person
3,Ro_10_10_05,Associate
4,Ro_10_10_12,Chairman
...,...,...
245,Ro_70_50_48,Locksmith
246,Ro_70_50_60,Painter and decorator
247,Ro_70_50_61,Partition installer
248,Ro_70_50_62,Paving and surfacing installer


Build the query text per Uniphi row

In [13]:
def build_query_text(row):
    parts = []
    
    if pd.notna(row["Classification"]) and row["Classification"].strip():
        parts.append(f"Classification: {row['Classification'].strip()}")
        
    if pd.notna(row["Discipline"]) and row["Discipline"].strip():
        parts.append(f"Discipline: {row['Discipline'].strip()}")
        
    if pd.notna(row["Position"]) and row["Position"].strip():
        parts.append(f"Position: {row['Position'].strip()}")
        
    return " | ".join(parts)

# Apply it
df_u["query_text"] = df_u.apply(build_query_text, axis=1)

# Check
print(df_u["query_text"].iloc[0])
df_u=df_u[["Classification","Discipline","Position","query_text"]]
df_u.head()

Classification: Acradis PAA Salary | Discipline: Design Management | Position: Design Manager - North


,Classification,Discipline,Position,query_text
2,Acradis PAA Salary,Design Management,Design Manager - North,Classification: Acradis PAA Salary | Disciplin...
3,Acradis PAA Salary,Project Management,Digital Engineering Discipline Lead,Classification: Acradis PAA Salary | Disciplin...
4,Acradis PAA Salary,Project Management,Innovation & Value Engineering Lead,Classification: Acradis PAA Salary | Disciplin...
5,Acradis PAA Salary,Design Management,AMT Design Lead,Classification: Acradis PAA Salary | Disciplin...
6,Acradis PAA Salary,Building Structures,Buildings Structures Lead,Classification: Acradis PAA Salary | Disciplin...


Stage-1 (ML): Bi-encoder retrieval (fast, scalable)
Encode all Uniclass titles once.

Encode all queries.

Do cosine similarity search to get top-k candidates per row.

In [18]:
from sentence_transformers import SentenceTransformer, util
bi_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

titles = df_c["UniclassTitle"].tolist()
codes  = df_c["UniclassCode"].tolist()

# Encode once
emb_titles = bi_model.encode(titles, convert_to_tensor=True, normalize_embeddings=True)
emb_queries = bi_model.encode(df_u["query_text"].tolist(), convert_to_tensor=True, normalize_embeddings=True)
TOP_K = 10
bi_hits = util.semantic_search(emb_queries, emb_titles, top_k=TOP_K)

Stage-2 (ML): Cross-encoder re-ranking (precision boost)

In [19]:
# Small, strong cross-encoder (BERT) for pair scoring
from sentence_transformers import CrossEncoder


ce_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank_for_row(q_text, candidate_indices):
    pairs = [(q_text, titles[i]) for i in candidate_indices]
    ce_scores = ce_model.predict(pairs)  # higher = better
    best_local = int(np.argmax(ce_scores))
    return candidate_indices[best_local], float(ce_scores[best_local]), ce_scores


c:\Users\SuratAdhikari\Desktop\classfication\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SuratAdhikari\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not ins

Put it together → pick best match + return scores

In [ ]:
best_title, best_code = [], []
bi_score_top, ce_score_best = [], []
alts = []

for i, hits in enumerate(bi_hits):
    if not hits:
        best_title.append(""); best_code.append("")
        bi_score_top.append(0.0); ce_score_best.append(0.0); alts.append([])
        continue

    cand_idxs = [h["corpus_id"] for h in hits]
    cand_bi   = [float(h["score"]) for h in hits]

    # Re-rank with cross-encoder, keep the best
    idx, ce_s, ce_scores = rerank_for_row(df_u.iloc[i]["query_text"], cand_idxs)

    best_title.append(titles[idx])
    best_code.append(codes[idx])
    bi_score_top.append(max(cand_bi))
    ce_score_best.append(ce_s)

    # keep top-3 candidates (by cross-encoder) for auditing
    order = np.argsort(-ce_scores)
    alt_list = []
    for j in order[1:3]:
        alt_idx = cand_idxs[j]
        alt_list.append({
            "title": titles[alt_idx],
            "code":  codes[alt_idx],
            "ce_score": float(ce_scores[j])
        })
    alts.append(alt_list)

df_ml = df_u.copy()
df_ml["MatchedUniclassTitle"] = best_title
df_ml["MatchedUniclassCode"]  = best_code
df_ml["BiEncScore"]           = [round(x, 4) for x in bi_score_top]  # cosine 0..1
df_ml["CrossEncScore"]        = [round(x, 4) for x in ce_score_best] # uncalibrated (use relatively)
df_ml["Alternates_CE_Top3"]   = alts

# Simple confidence rule (start here; tune after manual review)
df_ml["IsConfident"] = (df_ml["BiEncScore"] >= 0.60) & (df_ml["CrossEncScore"] >= 0.55)

df_ml.head(10)

# df_ml.to_csv("")


',Classification,Discipline,Position,query_text,MatchedUniclassTitle,MatchedUniclassCode,BiEncScore,CrossEncScore,Alternates_CE_Top3,IsConfident\r\n2,Acradis PAA Salary,Design Management,Design Manager - North,Classification: Acradis PAA Salary | Discipline: Design Management | Position: Design Manager - North,Design manager,Ro_50_10_22,0.5543,-3.6704,"[{\'title\': \'Design roles\', \'code\': \'Ro_50\', \'ce_score\': -6.777615547180176}, {\'title\': \'Building design roles\', \'code\': \'Ro_50_10\', \'ce_score\': -7.7109551429748535}]",False\r\n3,Acradis PAA Salary,Project Management,Digital Engineering Discipline Lead,Classification: Acradis PAA Salary | Discipline: Project Management | Position: Digital Engineering Discipline Lead,Digital engineer,Ro_50_20_26,0.5235,-3.7867,"[{\'title\': \'Project manager\', \'code\': \'Ro_30_10_67\', \'ce_score\': -5.574281215667725}, {\'title\': \'Electronics engineer\', \'code\': \'Ro_50_20_31\', \'ce_score\': -7.622633934020996}]",False\r\n4,Acra

In [22]:
df_ml.to_csv("final.csv")